In [1]:
import re

from google.api_core.operation import Operation
from google.cloud import documentai_v1 as documentai
from google.cloud import storage
import shutil
import os
from datetime import date, datetime
from os import chdir, getcwd, listdir, remove
from os.path import isfile

In [2]:
# Credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/home/ilicio/projetos/chaves/estudos-ilicio-cbb35bf6f539.json'

In [21]:
# Copy object bucket to my bucket
#gsutil cp gs://cloud-samples-data/documentai/codelabs/ocr/Winnie_the_Pooh.pdf gs://ocr_bucket_two/

In [3]:
#função para gerar os logs
def logaction(texto):
    dateTimeObj = datetime.now()
    timestampStr = dateTimeObj.strftime("%d-%b-%Y %H")
    timestampStr2 = dateTimeObj.strftime("%d-%b-%Y %H:%M:%S.%f")
    logtxt = str(timestampStr2)+"-"+texto

    try:        
     
        nomearq = "/home/ilicio/projetos/"+"LOGIMPORT_"+str(timestampStr)+".txt"
        arquivo = open(nomearq, "a")
        frases = list()
        #insere data e hora
        frases.append(timestampStr2)
        frases.append(texto+"\n")
        arquivo.writelines(frases)
        arquivo.close()
    except FileNotFoundError:
        arquivo = open(nomearq, 'w+')
        #arquivo.writelines(u'Arquivo criado pois nao existia')
        arquivo = "/home/ilicio/projetos/"+"LOGIMPORT_"+str(timestampStr)+".txt"
        arquivo = open(arquivo, "a")
        frases = list()
        #insere data e hora
        frases.append(timestampStr2)
        frases.append(texto+"\n")
        arquivo.writelines(frases)
        arquivo.close()

# if __name__=='__main__':
#     logaction("teste4")
#     logaction("teste5")
#     logaction("teste6")
#     logaction("teste7")

In [4]:
def batch_process_documents(
    project_id: str,
    location: str,
    processor_id: str,
    gcs_input_uri: str,
    input_mime_type: str,
    gcs_output_uri: str,
) -> Operation:
    """
    Constructs a request to process a document using the Document AI
    Asynchronous API.
    """
    # You must set the api_endpoint if you use a location other than 'us', e.g.:
    opts = {}
    if location == "eu":
        opts = {"api_endpoint": "eu-documentai.googleapis.com"}

    # Instantiates a client
    documentai_client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the processor, e.g.:
    # projects/project-id/locations/location/processor/processor-id
    # You must create new processors in the Cloud Console first
    resource_name = documentai_client.processor_path(project_id, location, processor_id)

    # Cloud Storage URI for the Input Document
    input_document = documentai.GcsDocument(
        gcs_uri=gcs_input_uri, mime_type=input_mime_type
    )

    # Load GCS Input URI into a List of document files
    input_config = documentai.BatchDocumentsInputConfig(
        gcs_documents=documentai.GcsDocuments(documents=[input_document])
    )

    # Cloud Storage URI for Output directory
    gcs_output_config = documentai.DocumentOutputConfig.GcsOutputConfig(
        gcs_uri=gcs_output_uri
    )

    # Load GCS Output URI into OutputConfig object
    output_config = documentai.DocumentOutputConfig(gcs_output_config=gcs_output_config)

    # Configure Process Request
    request = documentai.BatchProcessRequest(
        name=resource_name,
        input_documents=input_config,
        document_output_config=output_config,
    )

    # Future for long-running operations returned from Google Cloud APIs.
    operation = documentai_client.batch_process_documents(request)

    return operation

In [5]:
def get_documents_from_gcs(
    gcs_output_uri: str, operation_name: str
) -> [documentai.Document]:
    """
    Download the document output from GCS.
    """
    print('output_uri:{}  operation_name:{}'.format(gcs_output_uri, operation_name))

    # The GCS API requires the bucket name and URI prefix separately
    match = re.match(r"gs://([^/]+)/(.+)", gcs_output_uri)
    print('chegou ate aqui {}'.format(match))
    output_bucket = match.group(1)
    prefix = match.group(2)

    # The output files will be in a new subdirectory with the Operation ID as the name
    operation_id = re.search("operations\/(\d+)", operation_name, re.IGNORECASE).group(1)

    output_directory = f"{prefix}/{operation_id}"

    storage_client = storage.Client()

    # List of all of the files in the directory `gs://gcs_output_uri/operation_id`
    blob_list = list(storage_client.list_blobs(output_bucket, prefix=output_directory))

    output_documents = []

    for blob in blob_list:
        # Document AI should only output JSON files to GCS
        if ".json" in blob.name:
            document = documentai.types.Document.from_json(blob.download_as_bytes())
            output_documents.append(document)
        else:
            print(f"Skipping non-supported file type {blob.name}")

    return output_documents

In [6]:
def main():
    """
    Run the codelab.
    """

    project_id = 'estudos-ilicio'
    location = 'eu'  # Format is 'us' or 'eu'
    processor_id = '15cc86a7e7473fc0'  # Create processor in Cloud Console

    # Format 'gs://input_bucket/directory/file.pdf'
    gcs_input_uri = "gs://ocr_bucket_two/Winnie-the-Pooh30.pdf"
    input_mime_type = "application/pdf"

    # Format 'gs://output_bucket/directory'
    gcs_output_uri = "gs://ocr_bucket_two/ilicio"

    # Batch Process returns a Long Running Operation (LRO)
    operation = batch_process_documents(
        project_id=project_id,
        location=location,
        processor_id=processor_id,
        gcs_input_uri=gcs_input_uri,
        input_mime_type=input_mime_type,
        gcs_output_uri=gcs_output_uri,
    )

    # Format: projects/PROJECT_NUMBER/locations/LOCATION/operations/OPERATION_ID
    operation_name = operation.operation.name

    # Continually polls the operation until it is complete.
    # This could take some time for larger files
    print(f"Waiting for operation {operation_name} to complete...")
    result = operation.result(timeout=3600)

    # NOTE: Can also use callbacks for asynchronous processing
    #
    # def my_callback(future):
    #   result = future.result()
    #
    # operation.add_done_callback(my_callback)

    print("Document processing complete.")

    # Get the Document Objects from the Output Bucket
    document_list = get_documents_from_gcs(
        gcs_output_uri=gcs_output_uri, operation_name=operation_name
    )

    for document in document_list:
        logaction(document.text)
        print(document.text)

In [7]:
# Execute main function
main()

Waiting for operation projects/35175528436/locations/eu/operations/16441799757702191666 to complete...
Document processing complete.
output_uri:gs://ocr_bucket_two/ilicio  operation_name:projects/35175528436/locations/eu/operations/16441799757702191666
chegou ate aqui <re.Match object; span=(0, 26), match='gs://ocr_bucket_two/ilicio'>
This is a reproduction of a library book that was digitized
by Google as part of an ongoing effort to preserve the
information in books and make it universally accessible.
Google books
https://books.google.com
TM
Googl
OF
University of
Michigan
Libraries
AB
1817
2
SCIENTIA VE ITAS
ITAS
Digitized by
Googk
I
Digitized by
Google
Digitized by
Google
i
I
N
I
1
7
P
}
Digitized by
Google
Digitized by Google
WINNIE-THE-POOH
BY A. A. MILNE
with decorations
by Ernest H. Shepard
PUBLISHED BY
E. P. DUTTON & CO., INC., NEW YORK
123
Digitized by
Google
Juvenile
Collection
1
};
WINNIE-THE-POOH
COPYRIGHT, 1926, BY E. P. DUTTON & CO., INC.
COPYRIGHT RENEWAL, 1954, BY A. A